## CSS Project Part 2
Arathy Pillai, Alexander Guth, Hammad Akthar, Hiba Saleel and Julian Jetz

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from string import punctuation

In [2]:
plot_size = plt.rcParams["figure.figsize"]  
plot_size[0] = 15  
plot_size[1] = 8  
plt.rcParams["figure.figsize"] = plot_size  

### Load data and determine na values
repost and Text are determined as na values, because they ... **[TODO]**

In [3]:
na_values = ['nan', 'N/A', 'NaN', 'NaT', '', 'Text', 'repost', 'r e p o s t -', 'Repost']
#na_values = ['nan', 'N/A', 'NaN', 'NaT', '0', '', 'Text', 'repost', 'r e p o s t -', 'Repost']
ira_data = pd.read_csv('../Data/data_IRA_Ads.csv', sep=";", parse_dates=['AD_CREATION_DATE', 'AD_END_DATE'], 
                       dayfirst=False, index_col='AD_ID', na_values=na_values)
ira_data = ira_data[ira_data['AD_IMPRESSIONS']>0]

### Click per View Ratio
#Clicks / #Impressions

In [4]:
ira_data.insert(5, 'AD_CLICKS_PER_VIEW', ira_data['AD_CLICKS']/ira_data['AD_IMPRESSIONS'])

In [5]:
epoch = datetime.datetime.utcfromtimestamp(0)

def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

In [6]:
ira_data['AD_CREATION_WEEKDAY'] = ira_data['AD_CREATION_DATE'].dt.dayofweek
ira_data['AD_CREATION_YEAR'] = ira_data['AD_CREATION_DATE'].dt.year
ira_data['AD_CREATION_MONTH'] = ira_data['AD_CREATION_DATE'].dt.month
ira_data['AD_CREATION_DAY'] = ira_data['AD_CREATION_DATE'].dt.day
ira_data['AD_CREATION_TIME'] = ira_data['AD_CREATION_DATE'].dt.time
ira_data['AD_CREATION_EPOCH_TIME'] = ira_data['AD_CREATION_DATE'].apply(lambda x: unix_time_millis(x))))

SyntaxError: invalid syntax (<ipython-input-6-2ae59004a0f8>, line 6)

### Convert ruble
Removes "RUB" from the cell AD_SPEND

In [ ]:
ira_data['AD_SPEND']=ira_data['AD_SPEND'].str.replace(' RUB','')

### Drop all subsets, where AD_TEXT is na

In [ ]:
ira_data=ira_data.dropna(subset=['AD_TEXT']);
ira_data['AD_TEXT'].count()

### Remove hyperlinks

In [ ]:
ira_data['AD_TEXT']=ira_data['AD_TEXT'].str.replace(r'https?:\/\/.*[\r\n]*', '')
ira_data['AD_TEXT']=ira_data['AD_TEXT'].str.replace(r'http?:\/\/.*[\r\n]*', '')

### Select all duplicate ad texts

In [ ]:
duplicateDF = pd.concat(g for _, g in ira_data.groupby("AD_TEXT") if len(g) > 1).copy()
duplicateDF.sort_values(by=['AD_TEXT'], inplace=True)

In [ ]:
ira_data['AD_TEXT'].count()

In [ ]:
uniqueDF = ira_data.copy()
uniqueDF.drop_duplicates(subset ='AD_TEXT', 
                     keep = False, inplace = True)

### Total Duplicates

In [ ]:
ira_data['AD_TEXT'].count()

In [ ]:
duplicateDF['AD_TEXT'].count()

In [ ]:
uniqueDF['AD_TEXT'].count()

### Unique Duplicate Pairs

In [ ]:
duplicateDF['AD_TEXT'].nunique()

### Occurence of Duplicate Pairs

In [ ]:
duplicateDF['AD_TEXT'].value_counts().values

In [ ]:
duplicateDF[duplicateDF['AD_IMPRESSIONS']>0]['AD_TEXT'].value_counts()

In [ ]:
duplicateDF[(duplicateDF['AD_IMPRESSIONS']>0) & duplicateDF['AD_TEXT'].str.match('We speak for all fellow members of LGBT community across the nation.')][['AD_CLICKS', 'AD_IMPRESSIONS', 'AD_CREATION_DATE', 'AD_CREATION_WEEKDAY','AD_CREATION_YEAR', 'AD_CREATION_MONTH','AD_CREATION_DAY','AD_CREATION_TIME','AD_TEXT', 'AD_SPEND', 'AD_POTENTIAL_TARGET']].head(20)

In [ ]:
duplicateDF.sample(25)

In [ ]:
uniqueDF.sample(25)

In [ ]:
corrDf = ira_data[[ 'AD_CLICKS', 'AD_IMPRESSIONS', 'AD_CLICKS_PER_VIEW', 'AD_POTENTIAL_TARGET','AD_CREATION_WEEKDAY', 'AD_CREATION_EPOCH_TIME', 'AD_CREATION_DATE', 'AD_CREATION_YEAR', 'AD_CREATION_MONTH', 'AD_CREATION_DAY', 'AD_CREATION_TIME']]
corrDf = corrDf.dropna()

In [ ]:
scatter_matrix(corrDf, figsize=(12, 12))
corrDf.corr()

In [ ]:
corrUniqueDf = uniqueDF[[ 'AD_CLICKS', 'AD_IMPRESSIONS', 'AD_CLICKS_PER_VIEW', 'AD_POTENTIAL_TARGET','AD_CREATION_WEEKDAY', 'AD_CREATION_EPOCH_TIME', 'AD_CREATION_DATE', 'AD_CREATION_YEAR', 'AD_CREATION_MONTH', 'AD_CREATION_DAY', 'AD_CREATION_TIME']]
corrUniqueDf = corrUniqueDf.dropna()

In [ ]:
scatter_matrix(corrUniqueDf, figsize=(12, 12))
corrUniqueDf.corr()

In [ ]:
corrDuplicateDf = duplicateDF[[ 'AD_CLICKS', 'AD_IMPRESSIONS', 'AD_CLICKS_PER_VIEW', 'AD_POTENTIAL_TARGET','AD_CREATION_WEEKDAY', 'AD_CREATION_EPOCH_TIME', 'AD_CREATION_DATE', 'AD_CREATION_YEAR', 'AD_CREATION_MONTH', 'AD_CREATION_DAY', 'AD_CREATION_TIME']]
corrDuplicateDf = corrDuplicateDf.dropna()

In [ ]:
scatter_matrix(corrDuplicateDf, figsize=(12, 12))
corrDuplicateDf.corr()